In [1]:
# 4-thrust.py
import math

thresh = 0.2

class FourThrustSimulator:
    _instance = None
    
    def __init__(self, initial_angle=0, initial_roll=0, initial_yaw=0, speed=1):
        # Initial angles and angular velocities for pitch, roll, and yaw
        self.angle = initial_angle / 50
        self.angular_velocity = 0.0
        self.gravity = 9.81
        self.length = 5.0  # Length in meters
        self.base_damping = 0.05
        self.dt = 1
        self.time = 0
        self.speed = speed

        # Separate angular values for pitch, roll, and yaw
        self.pitch = initial_angle / 50
        self.pitch_velocity = 0.0

        self.roll = initial_roll / 50
        self.roll_velocity = 0.0

        self.yaw = initial_yaw
        self.yaw_velocity = 0.0

        # Initial thrust values
        self.front_left_thrust = 0
        self.front_right_thrust = 0
        self.back_left_thrust = 0
        self.back_right_thrust = 0
        
    def psi_to_newtons(self, psi):
        """Convert PSI to Newtons (approximate conversion)"""
        # 1 PSI ≈ 6.89476 kPa
        # Force = Pressure * Area (assuming 1 square inch nozzle)
        # 1 square inch = 0.00064516 square meters
        return (psi * 6.89476 * 0.00064516 * 1000) / 300  # Convert to Newtons

    def calculate_damping(self, angular_velocity):
        velocity_factor = abs(angular_velocity) * 0.1
        time_factor = min(0.1, self.time * 0.0001)
        return self.base_damping * (1 + velocity_factor + time_factor)

    def calculate_acceleration(self, angle, angular_velocity):
        """Calculate the angular acceleration due to gravity and damping for pitch or roll."""
        angle_rad = math.radians(angle)
        angular_acceleration = -(self.gravity / self.length) * math.sin(angle_rad)
        
        # Apply damping based on the angular velocity
        current_damping = self.calculate_damping(angular_velocity)
        angular_acceleration -= current_damping * angular_velocity
        
        return angular_acceleration
    
    def apply_thrust(self, front_left_psi=0, front_right_psi=0, back_left_psi=0, back_right_psi=0):
        """Apply thrust to each of the four thrusters (in PSI) and update pitch, roll, and yaw changes based on the imbalance."""
        # Store thrust values for reference in the update loop
        self.front_left_thrust = self.psi_to_newtons(front_left_psi)
        self.front_right_thrust = self.psi_to_newtons(front_right_psi)
        self.back_left_thrust = self.psi_to_newtons(back_left_psi)
        self.back_right_thrust = self.psi_to_newtons(back_right_psi)

        # Calculate net forces for pitch, roll, and yaw based on thrust imbalances
        pitch_force = (self.back_left_thrust + self.back_right_thrust) - (self.front_left_thrust + self.front_right_thrust)
        roll_force = (self.front_right_thrust + self.back_right_thrust) - (self.front_left_thrust + self.back_left_thrust)
        yaw_force = (self.front_left_thrust + self.back_right_thrust) - (self.front_right_thrust + self.back_left_thrust)

        # Convert forces to angular accelerations
        pitch_acceleration = pitch_force * 0.01  # Adjust 0.01 as a sensitivity factor for pitch
        roll_acceleration = roll_force * 0.01   # Adjust 0.01 as a sensitivity factor for roll
        yaw_acceleration = yaw_force * 0.005    # Adjust 0.005 as a sensitivity factor for yaw

        # Update velocities with the calculated accelerations
        self.pitch_velocity += pitch_acceleration * self.dt
        self.roll_velocity += roll_acceleration * self.dt
        self.yaw_velocity += yaw_acceleration * self.dt

    def update(self):
        # Update pitch with gravity and damping effects
        pitch_acceleration = self.calculate_acceleration(self.pitch, self.pitch_velocity)
        self.pitch_velocity += pitch_acceleration * self.dt * self.speed
        self.pitch += self.pitch_velocity * self.dt * self.speed
        
        # Update roll with gravity and damping effects
        roll_acceleration = self.calculate_acceleration(self.roll, self.roll_velocity)
        self.roll_velocity += roll_acceleration * self.dt * self.speed
        self.roll += self.roll_velocity * self.dt * self.speed

        # Apply damping to yaw to naturally bring it to zero without a pendulum effect
        yaw_damping = self.calculate_damping(self.yaw_velocity)
        self.yaw_velocity -= yaw_damping * self.yaw_velocity
        self.yaw += self.yaw_velocity * self.dt * self.speed

        # Ensure the angles are within a reasonable range to prevent excessive tilting
        self.pitch = max(min(self.pitch, 50), -50)
        self.roll = max(min(self.roll, 50), -50)
        
        self.time += self.dt

    def is_stable(self):
        """Check if the drone is stable (near zero angles and velocities)."""
        return (abs(self.pitch) < thresh and abs(self.pitch_velocity) < thresh and
                abs(self.roll) < thresh and abs(self.roll_velocity) < thresh and
                abs(self.yaw) < thresh and abs(self.yaw_velocity) < thresh)
    
    # IMU-like getter functions
    def get_ypr(self):
        """Get Yaw, Pitch, Roll values (similar to IMU)"""
        return [self.yaw, self.pitch, self.roll]
    
    def get_motion_data(self):
        """Get detailed motion data"""
        return {
            'pitch': self.pitch,
            'pitch_velocity': self.pitch_velocity,
            'roll': self.roll,
            'roll_velocity': self.roll_velocity,
            'yaw': self.yaw,
            'yaw_velocity': self.yaw_velocity,
            'is_stable': self.is_stable()
        }

    def reset(self, initial_pitch=0, initial_roll=0, initial_yaw=0, speed=1):
        """Reset the simulator to its initial state with specified pitch, roll, and yaw."""
        self.pitch = initial_pitch / 50
        self.roll = initial_roll / 50
        self.yaw = initial_yaw
        self.angle = initial_pitch / 50
        self.angular_velocity = 0.0
        self.time = 0
        self.speed = speed
        # Reset thrust values
        self.front_left_thrust = 0
        self.front_right_thrust = 0
        self.back_left_thrust = 0
        self.back_right_thrust = 0

    @classmethod
    def get_instance(cls, initial_angle=0, speed=1):
        if cls._instance is None:
            cls._instance = FourThrustSimulator(initial_angle, speed)
        return cls._instance

def get_simulated_angle(max_steps=300):
    simulator = FourThrustSimulator.get_instance()
    if not simulator.is_stable() and max_steps > 0:
        simulator.update()
    return simulator.get_motion_data()

In [2]:
psi = 60

# Thrust control function
def thrust_control(correction: float):
    if correction == 0:
        return (0, 0)  # Both thrusters off
    elif correction > 0:
        return (psi, 0)  # Left thruster on, right off
    else:
        return (0, psi)  # Right thruster on, left off

# Correction function (PID control)
def calculate_correction(pitch: float, previous_error: float, integral: float, dt: float, kp: float, ki: float, kd: float):
    error = pitch  # The error is simply the current pitch angle
    integral += error * dt  # Integral is the sum of all past errors over time
    derivative = (error - previous_error) / dt  # Derivative is the rate of change of the error
    correction = kp * error + ki * integral + kd * derivative
    return correction, error, integral  # Return the correction and updated error and integral for the next loop

# PID Constants
kp = 100
ki = 0.1
kd = 0.1

# Initialize variables
previous_error = 0
integral = 0
dt = 0.1 

pitch = 10

# Calculate the correction using PID
correction, previous_error, integral = calculate_correction(pitch, previous_error, integral, dt, kp, ki, kd)

# Control the thrusters based on the correction value
left_thrust, right_thrust = thrust_control(correction)

# Print the thrust control results
print(f"Left Thrust: {left_thrust}, Right Thrust: {right_thrust}")

Left Thrust: 60, Right Thrust: 0


In [51]:
import numpy as np
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go

# Initialize the app
app = dash.Dash(__name__)
graphid = "drone-graph"

# Create a layout with a graph, interval, and controls
app.layout = html.Div([
     html.Div([
        html.Button('Reset', id='reset-button', n_clicks=0),
        dcc.Input(
            id='initial-pitch',
            type='number',
            min=-50,
            max=50,
            placeholder='Initial Pitch Angle (-50 to 50)',
            value=10,
            style={'margin': '10px'}
        ),
        dcc.Input(
            id='initial-roll',
            type='number',
            min=-50,
            max=50,
            placeholder='Initial Roll Angle (-50 to 50)',
            value=0,
            style={'margin': '10px'}
        ),
        dcc.Input(
            id='initial-yaw',
            type='number',
            min=-50,
            max=50,
            placeholder='Initial Yaw Angle (-50 to 50)',
            value=0,
            style={'margin': '10px'}
        ),
        html.Div([
            html.Label('Front Left Thruster (PSI):'),
            dcc.Input(
                id='front-left-thrust',
                type='number',
                min=0,
                max=100,
                value=0,
                style={'margin': '10px'}
            ),
            html.Label('Front Right Thruster (PSI):'),
            dcc.Input(
                id='front-right-thrust',
                type='number',
                min=0,
                max=100,
                value=0,
                style={'margin': '10px'}
            ),
            html.Label('Back Left Thruster (PSI):'),
            dcc.Input(
                id='back-left-thrust',
                type='number',
                min=0,
                max=100,
                value=0,
                style={'margin': '10px'}
            ),
            html.Label('Back Right Thruster (PSI):'),
            dcc.Input(
                id='back-right-thrust',
                type='number',
                min=0,
                max=100,
                value=0,
                style={'margin': '10px'}
            ),
        ]),
    ], style={'margin': '10px'}),
    
    dcc.Graph(id=graphid),
    dcc.Interval(
        id='update-interval',
        interval=20,  # Update every Xms
        n_intervals=0
    )
])

# Initialize the simulator instance
simulator = FourThrustSimulator.get_instance()

@app.callback(
    Output(graphid, 'figure'),
    Input('update-interval', 'n_intervals'),
    Input('front-left-thrust', 'value'),
    Input('front-right-thrust', 'value'),
    Input('back-left-thrust', 'value'),
    Input('back-right-thrust', 'value'),
)

def update_drone_graph(n, front_left_thrust, front_right_thrust, back_left_thrust, back_right_thrust):

    # Apply thrust to the simulator
    simulator.apply_thrust(front_left_thrust, front_right_thrust, back_left_thrust, back_right_thrust)
    simulator.update()

    # Retrieve yaw, pitch, and roll from the simulator
    yaw, pitch, roll = simulator.get_ypr()

    # Define the vertices of the cube (representing the 4-sided base of the drone)
    length = 20
    cube_vertices = np.array([
        [-length, length, 0],  # Front Left
        [length, length, 0],   # Front Right
        [-length, -length, 0], # Back Left
        [length, -length, 0]   # Back Right
    ])

    # Apply rotation matrices for yaw, pitch, and roll using values from the simulator
    rotation_yaw = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])

    rotation_pitch = np.array([
        [1, 0, 0],
        [0, np.cos(pitch), -np.sin(pitch)],
        [0, np.sin(pitch), np.cos(pitch)]
    ])

    rotation_roll = np.array([
        [np.cos(roll), 0, np.sin(roll)],
        [0, 1, 0],
        [-np.sin(roll), 0, np.cos(roll)]
    ])

    # Apply the rotations
    rotated_vertices = cube_vertices @ rotation_yaw.T @ rotation_pitch.T @ rotation_roll.T

    # Separate x, y, and z values for plotting
    x_values = rotated_vertices[:, 0]
    y_values = rotated_vertices[:, 1]
    z_values = rotated_vertices[:, 2]
    
    traces = [
        # Cube edges (connecting the 4 vertices)
        go.Scatter3d(
            x=[x_values[0], x_values[1], x_values[3], x_values[2], x_values[0]],
            y=[y_values[0], y_values[1], y_values[3], y_values[2], y_values[0]],
            z=[z_values[0], z_values[1], z_values[3], z_values[2], z_values[0]],
            mode='lines',
            line=dict(color='blue', width=4),
            name='Drone Frame'
        ),
        # Add circles for thrusters at each corner of the cube
        go.Scatter3d(
            x=x_values,
            y=y_values,
            z=z_values,
            mode='markers+text',
            marker=dict(size=8, color='orange', symbol='circle'),
            text=[f'FL: {front_left_thrust} PSI', f'FR: {front_right_thrust} PSI',
                  f'BL: {back_left_thrust} PSI', f'BR: {back_right_thrust} PSI'],
            textposition='bottom center',
            name='Thrusters'
        )
    ]
    
    layout = go.Layout(
        scene=dict(
            xaxis=dict(range=[-50, 50], showgrid=True, zeroline=True),
            yaxis=dict(range=[-50, 50], showgrid=True, zeroline=True),
            zaxis=dict(range=[-50, 50], showgrid=True, zeroline=True),
            dragmode='orbit'  # Enable the drag mode for orbiting (dragging to rotate the plot)
        ),
        title='3D Cube Drone Simulation with Four Thrusters',
        showlegend=False,
        margin=dict(l=40, r=40, t=40, b=40)
    )
    
    return {'data': traces, 'layout': layout}

@app.callback(
    Output('update-interval', 'n_intervals'),
    Input('reset-button', 'n_clicks'),
    State('initial-pitch', 'value'),
    State('initial-roll', 'value'),
    State('initial-yaw', 'value')
)

def reset_simulation(n_clicks, initial_pitch, initial_roll, initial_yaw):
    if n_clicks > 0:
        simulator = FourThrustSimulator.get_instance()
        simulator.reset(initial_pitch=initial_pitch, initial_roll=initial_roll, initial_yaw=initial_yaw, speed=1)
    return 0

if __name__ == '__main__':
    app.run_server(debug=True)